In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
import warnings

# Set the display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)

student_data = pd.read_csv("/Users/srisatyaindukuri/Spider/ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv")
# Replace '-' with NaN
student_data.replace('-', np.nan, inplace=True)
student_data.sample(5)


/var/folders/1_/f10h6scn5x19_qnf0xw8fpyr0000gn/T/ipykernel_38156/1500055044.py:18: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  student_data = pd.read_csv("/Users/srisatyaindukuri/Spider/ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv")


,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE
191844,2990925,JU,Fl Comm Coll Transf-Upper Levl,201708,UG,SA,EPCH,Pre-Chemical Engineering,A,Asian,NaN,A Asian,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),4.47,29,29,33,22,28,650,700,1350,201708.0,86305.0,CHM,2210,NaN,A-
2407503,3695182,JU,Fl Comm Coll Transf-Upper Levl,202201,UG,EU,PEL,Elementary Education (Pre),A,Asian,NaN,A Asian,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,23,23,28,27,25,NaN,NaN,NaN,202208.0,88045.0,EDE,4504,NaN,A
1119574,3332050,BL,Beginner(FTIC) - Lower Level,201705,UG,SA,PUB,Public Health,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.02,24,24,27,26,25,NaN,NaN,NaN,201708.0,80456.0,MAC,1105,NaN,C
1912247,3551066,BL,Beginner(FTIC) - Lower Level,202005,UG,SA,BLG,Biology,B,Black or African American,Multi-Race,B Black or African American\nW White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.26,NaN,NaN,NaN,NaN,NaN,550,550,1100,202301.0,22876.0,DEP,2004,NaN,B
1105772,3329850,BL,Beginner(FTIC) - Lower Level,201801,UG,SA,MIC,Microbiology,H,Hispanic or Latino,NaN,NaN,Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.34,18,22,21,21,21,NaN,NaN,NaN,202001.0,18096.0,SYG,2000,NaN,A


In [3]:
# List of score columns
score_columns = ['ACTE', 'ACTM', 'ACTR', 'ACTS', 'EACT', 'SAT-ERW', 'SATM', 'SAT_TOTAL', 'Final_GPA']

# Convert score columns to numeric
for col in score_columns:
    student_data[col] = pd.to_numeric(student_data[col], errors='coerce')

In [4]:
# ACT to SAT conversion table
act_to_sat_conversion = {
    36: 1590, 35: 1540, 34: 1500, 33: 1460, 32: 1430, 31: 1400,
    30: 1370, 29: 1340, 28: 1310, 27: 1280, 26: 1240, 25: 1210,
    24: 1180, 23: 1140, 22: 1110, 21: 1080, 20: 1040, 19: 1010,
    18: 970, 17: 930, 16: 890, 15: 850, 14: 800, 13: 760,
    12: 710, 11: 670, 10: 630, 9: 590
}

# Convert EACT to SAT
def convert_act_to_sat(eact_score):
    if pd.isna(eact_score):
        return np.nan
    return act_to_sat_conversion.get(int(eact_score), np.nan)

# Apply the conversion to EACT scores
student_data['Converted_SAT'] = student_data['SAT_TOTAL']

# Identify where SAT_TOTAL is missing but EACT is available
mask = student_data['SAT_TOTAL'].isna() & student_data['EACT'].notna()

# Apply conversion
student_data.loc[mask, 'Converted_SAT'] = student_data.loc[mask, 'EACT'].apply(convert_act_to_sat)

# Step 4: Handle Remaining Missing Values

# Drop rows where Converted_SAT or Final_GPA is still NaN
datawithsat = student_data.dropna(subset=['Converted_SAT', 'Final_GPA'])

datawithsat.sample(5)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT
516781,3162901,BL,Beginner(FTIC) - Lower Level,201505,UG,SA,BSS,Social Science Education,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.47,23.0,28.0,31.0,27.0,27.0,NaN,NaN,NaN,201601.0,11331.0,MUH,3016,NaN,A+,1280.0
164500,2940396,UL,Other UG Transf - Lower Level,201408,UG,SA,UGU,Engineering,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.05,21.0,23.0,23.0,24.0,23.0,NaN,NaN,NaN,201505.0,51198.0,ACG,2021,NaN,C,1140.0
2033864,3573320,BL,Beginner(FTIC) - Lower Level,202008,UG,SA,CHM,Chemistry,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.62,32.0,26.0,32.0,24.0,29.0,580.0,550.0,1130.0,202201.0,11847.0,BSC,2086,NaN,A,1130.0
852467,3261008,JL,Fl Comm Coll Transf-Lower Levl,201808,UG,SA,BMS,Biomedical Sciences,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.07,21.0,23.0,20.0,20.0,21.0,580.0,520.0,1100.0,201808.0,88414.0,PCB,3063L,NaN,B+,1100.0
20384,1925745,JU,Fl Comm Coll Transf-Upper Levl,201501,UG,SP,PSY,Psychology,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.03,20.0,22.0,17.0,22.0,20.0,NaN,NaN,NaN,201508.0,87333.0,BSC,1020,NaN,B+,1040.0


In [5]:
grade_mapping = {
    # Standard Letter Grades
    'A': 4.0,
    'A+': 4.0,
    'A-': 3.67,  # Note: Some entries list A- as 3.67
    'B+': 3.5,   # Note: Some entries list B+ as 3.33
    'B': 3.0,
    'B-': 2.75,  # Note: Some entries list B- as 2.67
    'C+': 2.5,   # Note: Some entries list C+ as 2.33
    'C': 2.0,
    'C-': 1.75,  # Note: Some entries list C- as 1.67
    'D+': 1.5,   # Note: Some entries list D+ as 1.33
    'D': 1.0,
    'D-': 0.75,  # Note: Some entries list D- as 0.67
    'F': 0.0,
    'FF': 0.0,   # Dishonesty Failing Grade
    'MF': 0.0,   # Missing Failing Grade
    'IF': 0.0,   # Incomplete Failing Grade

    # Satisfactory/Unsatisfactory Grades
    'S': 0.0,    # In some systems, 'S' does not affect GPA
    'U': 0.0,

    # Pass/Fail Grades
    'P': 0.0,    # Pass (may not affect GPA)
    'NP': 0.0,   # No Pass

    # Withdrawal Grades
    'W': 0.0,
    'WE': 0.0,
    'WC': 0.0,
    'WF': 0.0,   # Withdraw Failing (may affect GPA)
    'WP': 4.0,  # Withdraw Passing

    # Incomplete Grades
    'I': None,
    'IA': None,
    'IA+': None,
    'IA-': None,
    'IB': None,
    'IB+': None,
    'IB-': None,
    'IC': None,
    'IC+': None,
    'IC-': None,
    'ID': None,
    'ID+': None,
    'ID-': None,
    'IP': None,
    'IS': None,
    'IU': None,
    'IX': None,
    'M': None,   # Missing Grade
    'MG': None,  # Missing Grade

    # Audit Grades
    'AU': None,
    'N': None,   # Audit or No Grade

    # Not Reported/No Grade
    'NR': None,
    'NG': None,  # No Grade
    'NC': None,  # No Credit

    # Administrative Grades
    'Z': None,   # Continuing Course
    'X': None,   # Deferred Grade

    # Special Cases and Other Grades
    'R': None,   # Repeat
    'IP': None,  # In Progress
    'S*': 0.0,
    'U*': 0.0,
    'W*': None,
    'I*': None,
    'Z*': None,
    'M*': None,

    # Grades with Special Characters Cleaned
    'A*': 4.0,
    'A#': 0.0,
    'A+*': 4.0,
    'A+*#': 0.0,
    'A-*': 3.75,
    'A-#': 0.0,
    'B*': 3.0,
    'B#': 0.0,
    'B+*': 3.5,
    'B+#': 0.0,
    'B-*': 2.75,
    'B-#': 0.0,
    'C*': 2.0,
    'C#': 0.0,
    'C+*': 2.5,
    'C+#': 0.0,
    'C-*': 1.75,
    'C-#': 0.0,
    'D*': 1.0,
    'D#': 0.0,
    'D+*': 1.5,
    'D+#': 0.0,
    'D-*': 0.75,
    'D-#': 0.0,
    'F*': 0.0,
    'F#': 0.0,
    'IF*': 0.0,
    'MF*': 0.0,

    # Competency-Based Grades (May not affect GPA)
    'CA+': None,
    'CA': None,
    'CA-': None,
    'CB': None,
    'CB+': None,
    'CB-': None,
    'CC': None,
    'CC+': None,
    'CC-': None,
    'CD': None,
    'CD+': None,
    'CD-': None,
    'CF': None,
    'CFF': None,
    'CI': None,
    'CIF': None,
    'CIS': None,
    'CM': None,
    'CMF': None,
    'CMU': None,
    'CN': None,
    'CS': None,
    'CU': None,
    'CW': None,
    'CX': None,
    'CZ': None,

    # Q Grades (Special Grading Scale)
    'Q1': 1.5,   # Equivalent to D+
    'Q2': 1.75,  # Equivalent to C-
    'Q3': 2.5,   # Equivalent to C+
    'Q4': 2.75,  # Equivalent to B-
    'Q5': 3.5,   # Equivalent to B+
    'Q6': 3.75,  # Equivalent to A-
    'QA': 4.0,   # Equivalent to A
    'QB': 3.0,   # Equivalent to B
    'QC': 2.0,   # Equivalent to C
    'QD': 1.0,   # Equivalent to D
    'QF': 0.0,   # Equivalent to F

    # Other Grades
    'IP': None,
    'IN': None,
    'NG*': None,
    'NR*': None,
    'S#': 0.0,
    'U#': 0.0,
    'W#': None,
    'X#': None,
    'Z#': None,

    # Repeat Indicators
    'R': None,
}

# Apply the mapping to the FINAL_GRADE column
datawithsat['FINAL_GRADE_NUM'] = datawithsat['FINAL_GRADE'].map(grade_mapping)
# Drop rows where 'FINAL_GRADE_NUM' is NaN
datawithsat = datawithsat.dropna(subset=['FINAL_GRADE_NUM']) #drop 32 rows

# Print the updated DataFrame
datawithsat.sample(5)

/var/folders/1_/f10h6scn5x19_qnf0xw8fpyr0000gn/T/ipykernel_38156/3412161981.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datawithsat['FINAL_GRADE_NUM'] = datawithsat['FINAL_GRADE'].map(grade_mapping)


,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM
236775,3043277,JU,Fl Comm Coll Transf-Upper Levl,201508,UG,SA,UGU,Engineering,H,Hispanic or Latino,NaN,NaN,Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.72,20.0,26.0,15.0,21.0,21.0,NaN,NaN,NaN,201601.0,10825.0,EGN,3365,NaN,B,1080.0,3.0
1126991,3333621,BL,Beginner(FTIC) - Lower Level,201708,UG,SA,EPCS,Pre-Computer Science,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.75,NaN,NaN,NaN,NaN,NaN,700.0,730.0,1430.0,201908.0,82148.0,ENC,3246,NaN,A,1430.0,4.0
2351528,3666222,BL,Beginner(FTIC) - Lower Level,202108,UG,SA,BCS,Computer Science,F,Non-Resident Alien,NaN,A Asian,Not Hispanic or Latino,F,Female,N,F,Foreign National,4.64,NaN,NaN,NaN,NaN,NaN,610.0,700.0,1310.0,202201.0,20035.0,PHY,2048,NaN,A,1310.0,4.0
1897010,3542890,BL,Beginner(FTIC) - Lower Level,202005,UG,SA,SBA,Studio Art,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.52,NaN,NaN,NaN,NaN,NaN,640.0,590.0,1230.0,202005.0,50422.0,ANT,2000,NaN,A,1230.0,4.0
978919,3284881,BL,Beginner(FTIC) - Lower Level,201608,UG,SA,PGB,General Business (Pre),W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.78,25.0,27.0,27.0,27.0,27.0,NaN,NaN,NaN,201801.0,23519.0,ACG,3103,NaN,D,1280.0,1.0


In [6]:
# Group by 'Pidm' (student ID)
grouped = datawithsat.groupby('Pidm')
grouped.head(10)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM
15,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201508.0,83267.0,NUR,3805,NaN,A+,1210.0,4.00
16,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201508.0,89943.0,NUR,3078,NaN,A+,1210.0,4.00
17,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201508.0,94441.0,NUR,4895,NaN,A+,1210.0,4.00
18,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201601.0,22277.0,NUR,4828C,NaN,A+,1210.0,4.00
19,794,SB,Second Baccalaureate,201508,UG,SA,PNR,Pre-Nursing,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.30,25.0,26.0,25.0,22.0,25.0,NaN,NaN,NaN,201601.0,22671.0,NUR,4128,NaN,A+,1210.0,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677342,3984725,UL,Other UG Transf - Lower Level,202405,UG,PH,HSS,Health Sciences,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.64,NaN,NaN,NaN,NaN,NaN,580.0,590.0,1170.0,202405.0,55256.0,SPC,2608,NaN,A-,1170.0,3.67
2677343,3984750,UL,Other UG Transf - Lower Level,202405,UG,BU,PAIS,Pre-Bus Analytics & Info Sys,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.29,18.0,17.0,18.0,20.0,18.0,410.0,340.0,750.0,202405.0,51968.0,MMC,3602,NaN,C+,750.0,2.50
2677344,3984772,BL,Beginner(FTIC) - Lower Level,202405,UG,AC,PSY,Psychology,F,Non-Resident Alien,NaN,NaN,NaN,F,Female,N,F,Foreign National,0.00,NaN,NaN,NaN,NaN,NaN,450.0,480.0,930.0,202405.0,54569.0,HUM,1020,NaN,F,930.0,0.00
2677400,3988409,UL,Other UG Transf - Lower Level,202405,UG,AC,PSY,Psychology,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.73,NaN,NaN,NaN,NaN,NaN,560.0,560.0,1120.0,202405.0,51082.0,PSY,2012,NaN,B,1120.0,3.00


In [10]:
all_data = []
# Process each student
for pidm, student_data in grouped:
    # Sort the student's data by 'Term'
    student_data = student_data.sort_values('Term')
    # Reset index to ensure correct iteration
    student_data = student_data.reset_index(drop=True)
    # Initialize an empty list to keep track of cumulative grades
    cumulative_grades = []
    for idx, row in student_data.iterrows():
        current_term = row['Term']
        # Get previous classes before the current term
        previous_classes = student_data[student_data['Term'] < current_term]
        # Compute cumulative GPA up to the current term
        if not previous_classes.empty:
            cumulative_gpa = previous_classes['FINAL_GRADE_NUM'].mean()
        else:
            cumulative_gpa = None  # No previous grades
        # Collect previous classes data
        previous_classes_list = previous_classes[['SUBJ', 'CRSE_NUMB', 'FINAL_GRADE', 'FINAL_GRADE_NUM']].to_dict('records')
        # Create a new row with current class data and previous classes info
        new_row = row.to_dict()
        new_row['Previous_Classes'] = previous_classes_list
        new_row['Cumulative_GPA'] = cumulative_gpa
        all_data.append(new_row)

# Create a new DataFrame from the list
new_df = pd.DataFrame(all_data)

# Display the new DataFrame
#print(new_df[['Pidm', 'Term', 'SUBJ', 'CRSE_NUMB', 'FINAL_GRADE', 'Cumulative_GPA', 'Previous_Classes']])
new_df.sample(5)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM,Previous_Classes,Cumulative_GPA
2199891,3731640,BL,Beginner(FTIC) - Lower Level,202208,UG,PH,HSS,Health Sciences,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.66,NaN,NaN,NaN,NaN,NaN,670.0,610.0,1280.0,202208.0,85534.0,GEY,4612,NaN,A,1280.0,4.0,[],NaN
1335177,3467218,UU,Other UG Transf - Upper Level,202101,UG,SA,LHS,Language-Speech-Hearing Sci,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.96,NaN,NaN,NaN,NaN,NaN,470.0,500.0,970.0,202108.0,90415.0,SPA,3112,NaN,B,970.0,3.0,"[{'SUBJ': 'SPA', 'CRSE_NUMB': '3261', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'SPA', 'CRSE_NUMB': '3030', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'SPA', 'CRSE_NUMB': '3002', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67}, {'SUBJ': 'SPA', 'CRSE_NUMB': '3101', 'FINAL_GRADE': 'D+', 'FINAL_GRADE_NUM': 1.5}, {'SUBJ': 'ASL', 'CRSE_NUMB': '2140C', 'FINAL_GRADE': 'C', 'FINAL_GRADE_NUM': 2.0}, {'SUBJ': 'SPA', 'CRSE_NUMB': '3310', 'FINAL_GRADE': 'C+', 'FINAL_GRADE_NUM': 2.5}...",3.048571
180715,3070772,UU,Other UG Transf - Upper Level,201608,UG,SA,PET,Physical Education,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.26,13.0,17.0,20.0,19.0,17.0,NaN,NaN,NaN,201808.0,86992.0,HSC,4624,NaN,B,930.0,3.0,"[{'SUBJ': 'DEP', 'CRSE_NUMB': '2004', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2085', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'ANT', 'CRSE_NUMB': '2410', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2085L', 'FINAL_GRADE': 'C', 'FINAL_GRADE_NUM': 2.0}, {'SUBJ': 'PAD', 'CRSE_NUMB': '4930', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'SYO', 'CRSE_NUMB': '4400', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {...",3.472381
1543264,3512451,BL,Beginner(FTIC) - Lower Level,201908,UG,SA,CHM,Chemistry,A,Asian,NaN,A Asian,Not Hispanic or Latino,M,Male,Y,C,Citizen (USA),4.20,NaN,NaN,NaN,NaN,NaN,740.0,780.0,1520.0,202008.0,80041.0,PCB,3023L,NaN,A,1520.0,4.0,"[{'SUBJ': 'CHM', 'CRSE_NUMB': '2046', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'SYG', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'MAC', 'CRSE_NUMB': '2241', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '2930', 'FINAL_GRADE': 'S', 'FINAL_GRADE_NUM': 0.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, ...",3.598235
1939160,3636938,BL,Beginner(FTIC) - Lower Level,202108,UG,SA,PFI,Finance (Pre),F,Non-Resident Alien,NaN,A Asian,Not Hispanic or Latino,F,Female,N,F,Foreign National,4.00,NaN,NaN,NaN,NaN,NaN,630.0,740.0,1370.0,202208.0,95654.0,ISM,3011,NaN,A,1370.0,4.0,"[{'SUBJ': 'ECO', 'CRSE_NUMB': '2023', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'ENC', 'CRSE_NUMB': '1101', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'MAC', 'CRSE_NUMB': '2233', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'EVR', 'CRSE_NUMB': '2001', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'CGS', 'CRSE_NUMB': '2100', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'QMB', 'CRSE_NUMB': '2100', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}...",4.000000


In [14]:
# Data preprocessing
data = new_df.dropna(subset=['FINAL_GRADE_NUM', 'Cumulative_GPA', 'Final_GPA', 'Converted_SAT'])
data = data.sort_values(by=['Pidm', 'Term'])
data.sample(5)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM,Previous_Classes,Cumulative_GPA
44356,2750736,FS,Former USF Student Returning,201905,UG,SM,BEE,Elementary Education,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,F,Female,Y,C,Citizen (USA),2.52,17.0,16.0,21.0,20.0,19.0,NaN,NaN,NaN,202101.0,26260.0,SLS,2122,NaN,A,1010.0,4.00,"[{'SUBJ': 'EDF', 'CRSE_NUMB': '3604', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'EVR', 'CRSE_NUMB': '2001', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'RED', 'CRSE_NUMB': '4333', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'RED', 'CRSE_NUMB': '4333', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'EDF', 'CRSE_NUMB': '3122', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'TSL', 'CRSE_NUMB': '4240', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67},...",3.212727
228760,3096930,BL,Beginner(FTIC) - Lower Level,201408,UG,SA,PMS,Pre-Medical Sciences,A,Asian,NaN,A Asian,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.68,26.0,33.0,30.0,27.0,29.0,NaN,NaN,NaN,201608.0,83938.0,PCB,3063L,NaN,A,1340.0,4.00,"[{'SUBJ': 'CHM', 'CRSE_NUMB': '2045L', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'MAC', 'CRSE_NUMB': '2241', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'SYG', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '4200', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'CHM', 'CRSE_NUMB': '2045', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, ...",3.700741
223530,3095100,BL,Beginner(FTIC) - Lower Level,201408,UG,SA,EGU,Engineering-General,A,Asian,Multi-Race,A Asian\nW White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.56,29.0,28.0,31.0,31.0,30.0,NaN,NaN,NaN,201501.0,16776.0,IDH,3100,NaN,A-,1370.0,3.67,"[{'SUBJ': 'EGN', 'CRSE_NUMB': '3000', 'FINAL_GRADE': 'S', 'FINAL_GRADE_NUM': 0.0}, {'SUBJ': 'ENC', 'CRSE_NUMB': '1102', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67}, {'SUBJ': 'MAC', 'CRSE_NUMB': '1147', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67}, {'SUBJ': 'CHS', 'CRSE_NUMB': '2440', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'MUL', 'CRSE_NUMB': '3011', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}]",3.140000
1394968,3487431,BL,Beginner(FTIC) - Lower Level,201908,UG,SA,EGS,Exploratory-Global & Social Sc,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.44,34.0,32.0,36.0,27.0,32.0,680.0,680.0,1360.0,202008.0,81216.0,CJL,3110,NaN,A,1360.0,4.00,"[{'SUBJ': 'INR', 'CRSE_NUMB': '3011', 'FINAL_GRADE': 'C', 'FINAL_GRADE_NUM': 2.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '1005', 'FINAL_GRADE': 'C+', 'FINAL_GRADE_NUM': 2.5}, {'SUBJ': 'CCJ', 'CRSE_NUMB': '3014', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'THE', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '2930', 'FINAL_GRADE': 'S', 'FINAL_GRADE_NUM': 0.0}, {'SUBJ': 'IDH', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {...",3.250833
651255,3266477,BL,Beginner(FTIC) - Lower Level,201605,UG,SA,PSY,Psychology,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),3.65,23.0,26.0,21.0,22.0,23.0,580.0,570.0,1150.0,201801.0,23710.0,SYO,3120,NaN,A,1150.0,4.00,"[{'SUBJ': 'AFA', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'HUM', 'CRSE_NUMB': '1020', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'SLS', 'CRSE_NUMB': '2901', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'PSY', 'CRSE_NUMB': '3213', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'ESC', 'CRSE

In [16]:
# Feature engineering
def create_features(df):
    df['Average_Prev_Grade'] = df.groupby('Pidm')['FINAL_GRADE_NUM'].shift().expanding().mean()
    df['Grades_Count'] = df.groupby('Pidm').cumcount()
    return df

In [17]:
data = create_features(data)
data.sample(5)

,Pidm,Admit_Code,Admit_Desc,Admit_Term,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,Final_GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Converted_SAT,FINAL_GRADE_NUM,Previous_Classes,Cumulative_GPA,Average_Prev_Grade,Grades_Count
2070792,3676897,BL,Beginner(FTIC) - Lower Level,202108,UG,SA,EPIE,Pre-Industrial Engineering,F,Non-Resident Alien,NaN,NaN,NaN,M,Male,N,F,Foreign National,3.51,NaN,NaN,NaN,NaN,NaN,600.0,700.0,1300.0,202201.0,18479.0,EGN,3000L,NaN,A-,1300.0,3.67,"[{'SUBJ': 'ENC', 'CRSE_NUMB': '1101', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'CHS', 'CRSE_NUMB': '2440', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'CHS', 'CRSE_NUMB': '2440L', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67}, {'SUBJ': 'MAC', 'CRSE_NUMB': '2311', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'PHI', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}]",3.434000,3.087465,6
1861499,3614820,BL,Beginner(FTIC) - Lower Level,202101,UG,SA,EPCS,Pre-Computer Science,W,White,NaN,W White,Not Hispanic or Latino,M,Male,Y,P,"PermResident, Asylee, Refugee",3.92,NaN,NaN,NaN,NaN,NaN,670.0,780.0,1450.0,202301.0,15181.0,PSY,2012,NaN,B+,1450.0,3.50,"[{'SUBJ': 'MAC', 'CRSE_NUMB': '2311', 'FINAL_GRADE': 'C', 'FINAL_GRADE_NUM': 2.0}, {'SUBJ': 'PHY', 'CRSE_NUMB': '2020', 'FINAL_GRADE': 'B', 'FINAL_GRADE_NUM': 3.0}, {'SUBJ': 'EGN', 'CRSE_NUMB': '3000', 'FINAL_GRADE': 'S', 'FINAL_GRADE_NUM': 0.0}, {'SUBJ': 'EGN', 'CRSE_NUMB': '3000L', 'FINAL_GRADE': 'B+', 'FINAL_GRADE_NUM': 3.5}, {'SUBJ': 'EGN', 'CRSE_NUMB': '4930', 'FINAL_GRADE': 'S', 'FINAL_GRADE_NUM': 0.0}, {'SUBJ': 'ENC', 'CRSE_NUMB': '1101', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67},...",2.430833,3.076250,13
2125467,3707528,BL,Beginner(FTIC) - Lower Level,202208,UG,AC,POL,Political Science,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.48,NaN,NaN,NaN,NaN,NaN,640.0,590.0,1230.0,202405.0,50868.0,POT,3003,NaN,A-,1230.0,3.67,"[{'SUBJ': 'GEA', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'PHI', 'CRSE_NUMB': '1600', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'EVR', 'CRSE_NUMB': '2001', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'CCJ', 'CRSE_NUMB': '3024', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'ENC', 'CRSE_NUMB': '1102', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'MAC', 'CRSE_NUMB': '1105', 'FINAL_GRADE': 'A-', 'FINAL_GRADE_NUM': 3.67},...",3.875625,3.090273,12
1925052,3634072,BL,Beginner(FTIC) - Lower Level,202108,UG,SA,BGY,Biology,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.54,26.0,24.0,29.0,24.0,26.0,630.0,590.0,1220.0,202201.0,21958.0,MHS,3063,NaN,A+,1220.0,4.00,"[{'SUBJ': 'MAC', 'CRSE_NUMB': '1147', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'MCB', 'CRSE_NUMB': '4905', 'FINAL_GRADE': 'S', 'FINAL_GRADE_NUM': 0.0}, {'SUBJ': 'GEY', 'CRSE_NUMB': '2000', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2932', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2010L', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '2010', 'FINAL_GRADE': 'A+', 'FINAL_GRADE_NUM': 4.0}...",3.428571,3.081189,5
78065,2878314,JU,Fl Comm Coll Transf-Upper Levl,201908,UG,SA,HLS,Health Sciences,H,Hispanic or Latino,NaN,NaN,Hispanic or Latino,M,Male,Y,C,Citizen (USA),3.82,NaN,NaN,NaN,NaN,NaN,640.0,600.0,1240.0,202005.0,57472.0,HSC,4430,NaN,A,1240.0,4.00,"[{'SUBJ': 'PCB', 'CRSE_NUMB': '3063', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'HSC', 'CRSE_NUMB': '4211', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'HSC', 'CRSE_NUMB': '4537', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'BSC', 'CRSE_NUMB': '3022', 'FINAL_GRADE': 'A', 'FINAL_GRADE_NUM': 4.0}, {'SUBJ': 'HSC', 'CRSE_NUMB': '2000', 'FINAL_

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [22]:
data = data.dropna(subset=['Average_Prev_Grade']) #drop 1 row

In [23]:
# Split data
students = data['Pidm'].unique()
train_students, test_students = train_test_split(students, test_size=0.2, random_state=42)
train_data = data[data['Pidm'].isin(train_students)]
test_data = data[data['Pidm'].isin(test_students)]

In [25]:
# Features and target
features = ['Average_Prev_Grade', 'Cumulative_GPA', 'Final_GPA', 'Converted_SAT', 'Grades_Count']
X_train = train_data[features]
y_train = train_data['FINAL_GRADE_NUM']
X_test = test_data[features]
y_test = test_data['FINAL_GRADE_NUM']

In [26]:
# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
# Model training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [28]:
# Prediction and evaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.8960840536457302


In [29]:
importances = model.feature_importances_
feature_names = features
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
print(feature_importance_df)






  

              Feature  Importance
0  Average_Prev_Grade    0.334008
1      Cumulative_GPA    0.272055
4        Grades_Count    0.177666
2           Final_GPA    0.119226
3       Converted_SAT    0.097045
